In [32]:
import json
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import ModelCheckpoint   

songs = []

full_genres = [['soundtrack'],
                ['jazz'],
                ['classical'],
               ['metal'],
               ['indietronica','wave','synth'],
               ['downtempo','trip hop'],
               ['edm','electronica','idm','dubstep','techno'],
               ['house'],
               ['r&b','rnb','soul'],
               ['rock'],
               ['hip hop','rap','trap','hiphop'],
               ['pop']]

def loadSongList(filename):
    with open(filename, 'r') as myfile:
        global songs
        data=myfile.read().replace('\n', '')
        song_data  = json.loads(data)
        songs = song_data    

def loadDataFrame(filename):
    global df
    df = pd.read_pickle(filename)

def loadModel(filename):
    global model
    model = load_model(filename)

def saveModel(filename):
    model.save(filename)


In [33]:
loadSongList('song_list_final.txt')
loadDataFrame('dataframe.pkl')

for i in range(10):
    print(songs[i])
df[:10]

print(len(songs))
print(len(df[0:]))

['Without My Heart', 'spotify:track:2mQJklbyeg532eqvoYgsRl', 'Tyrese', 'Black Rose', [8, 'r&b']]
['Blame It', 'spotify:track:08uGhvS5MfBk7crUCpnjva', 'Jamie Foxx', 'Intuition', [8, 'r&b']]
['Real Love', 'spotify:track:3XG801WWhqC753dekEBkMt', 'Mary J. Blige', "What's The 411?", [8, 'r&b']]
['AM // Radio', 'spotify:track:3cR19PDGb6w48V96kMaZOf', 'Earl Sweatshirt', "I Don't Like Shit, I Don't Go Outside: An Album by Earl Sweatshirt", [10, 'hip hop']]
['X (with 2 Chainz & Saudi)', 'spotify:track:4LmAnpjlhWTahvRkYR8xJa', 'ScHoolboy Q', 'Black Panther The Album Music From And Inspired By', [10, 'hip hop']]
['Opps (with Yugen Blakrok)', 'spotify:track:7bUcBztfGqO7cSI2gMZeCI', 'Vince Staples', 'Black Panther The Album Music From And Inspired By', [10, 'hip hop']]
['I Am', 'spotify:track:0DJBgBiYeSn6n1AXAkFVE8', 'Jorja Smith', 'Black Panther The Album Music From And Inspired By', [8, 'r&b']]
['Bloody Waters (with Anderson .Paak & James Blake)', 'spotify:track:4KXwFI9pgJLpUIAc9oSL8j', 'Ab-Soul'

In [34]:
processed_data = df[:]

processed_data['loudness'] = processed_data['loudness']/-20
processed_data['tempo'] = processed_data['tempo']/200
processed_data[:10]

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0.401,0.393,0.51485,0.0520,0.74700,0.000000,0.1210,0.279,0.593180,8.0
1,0.673,0.614,0.27130,0.1030,0.07820,0.000000,0.1390,0.362,0.880260,8.0
2,0.759,0.789,0.27825,0.0744,0.11200,0.000183,0.0526,0.811,0.475065,8.0
3,0.470,0.389,0.37660,0.0725,0.16200,0.007180,0.2250,0.524,0.411825,10.0
4,0.764,0.468,0.42400,0.2740,0.02250,0.000000,0.2600,0.360,0.655130,10.0
5,0.714,0.775,0.34055,0.3270,0.13700,0.000032,0.3960,0.840,0.640390,10.0
6,0.542,0.581,0.28765,0.2710,0.12500,0.000080,0.1850,0.298,0.368960,8.0
7,0.501,0.606,0.44300,0.4450,0.17600,0.000000,0.1580,0.194,0.902065,10.0
8,0.560,0.733,0.23320,0.3220,0.00111,0.000000,0.1620,0.413,0.408480,10.0
9,0.854,0.484,0.45270,0.2130,0.00763,0.000000,0.0970,0.302,0.650435,10.0


In [36]:
sample = np.random.choice(processed_data.index, size=int(len(processed_data)*0.9), replace=False)
train_data, test_data = processed_data.iloc[sample], processed_data.drop(sample)

print("Number of training samples is", len(train_data))
print("Number of testing samples is", len(test_data))

Number of training samples is 8645
Number of testing samples is 961


In [37]:
# Separate data and one-hot encode the output
# Note: We're also turning the data into numpy arrays, in order to train the model in Keras
features = np.array(train_data.drop('genre', axis=1))
targets = np.array(keras.utils.to_categorical(train_data['genre'], len(full_genres)))
features_test = np.array(test_data.drop('genre', axis=1))
targets_test = np.array(keras.utils.to_categorical(test_data['genre'], len(full_genres)))

print(features[:10])
print(targets[:10])

[[0.531      0.586      0.3005     0.21       0.28       0.
  0.0824     0.785      0.418555  ]
 [0.724      0.95       0.2833     0.0727     0.0028     0.0408
  0.174      0.725      0.62007   ]
 [0.794      0.868      0.2821     0.0547     0.688      0.882
  0.0971     0.455      0.57499   ]
 [0.886      0.585      0.1065     0.0493     0.0326     0.00000485
  0.0882     0.793      0.725035  ]
 [0.887      0.633      0.39905    0.183      0.00729    0.0000286
  0.257      0.429      0.519665  ]
 [0.486      0.563      0.6132     0.0334     0.0186     0.0000477
  0.074      0.678      0.52769   ]
 [0.426      0.408      0.47095    0.0524     0.771      0.00000788
  0.394      0.528      0.381545  ]
 [0.316      0.927      0.30225    0.0709     0.209      0.867
  0.102      0.576      0.848335  ]
 [0.523      0.364      0.6769     0.0645     0.614      0.731
  0.0816     0.795      0.58146   ]
 [0.89       0.633      0.27375    0.168      0.0232     0.000343
  0.0993     0.425      0.6

In [38]:
# Building the model
model = Sequential()
# model.add(Dense(2048, activation='relu', input_shape=(9,)))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(.2))

model.add(Dense(1000, activation='relu', input_shape=(9,)))
model.add(Dropout(.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))
# model.add(Dense(1000, activation='relu'))
# model.add(Dropout(.2))






model.add(Dense(len(full_genres), activation='softmax'))

# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 1000)              10000     
_________________________________________________________________
dropout_33 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_34 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_35 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 1000)              1001000   
__________

In [39]:
# Training the model
# model.fit(features, targets, epochs=200, batch_size=1000, verbose=1)

checkpointer = ModelCheckpoint(filepath='model.best.h5', 
                               verbose=1, save_best_only=True)

model.fit(features, targets, batch_size=3000, epochs=200,
          validation_split=0.2, 
#           callbacks=[checkpointer],
          verbose=1, shuffle=True)

Train on 6916 samples, validate on 1729 samples
Epoch 1/200
6916/6916 [==============================] - 8s 1ms/step - loss: 2.4353 - acc: 0.1575 - val_loss: 2.2886 - val_acc: 0.2163
Epoch 2/200
6916/6916 [==============================] - 0s 20us/step - loss: 2.2632 - acc: 0.2324 - val_loss: 2.1159 - val_acc: 0.2782
Epoch 3/200
6916/6916 [==============================] - 0s 20us/step - loss: 2.0650 - acc: 0.2817 - val_loss: 1.9910 - val_acc: 0.3112
Epoch 4/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.9506 - acc: 0.3135 - val_loss: 1.8713 - val_acc: 0.2961
Epoch 5/200
6916/6916 [==============================] - 0s 21us/step - loss: 1.8622 - acc: 0.3156 - val_loss: 1.8013 - val_acc: 0.3482
Epoch 6/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.7898 - acc: 0.3447 - val_loss: 1.7528 - val_acc: 0.3482
Epoch 7/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.7279 - acc: 0.3603 - val_loss: 1.6696 - val_acc: 0.4205
E

Epoch 120/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.1211 - acc: 0.5875 - val_loss: 1.4659 - val_acc: 0.4974
Epoch 121/200
6916/6916 [==============================] - 0s 19us/step - loss: 1.1215 - acc: 0.5863 - val_loss: 1.4846 - val_acc: 0.4939
Epoch 122/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.1220 - acc: 0.5857 - val_loss: 1.5181 - val_acc: 0.4887
Epoch 123/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.1311 - acc: 0.5817 - val_loss: 1.4696 - val_acc: 0.5049
Epoch 124/200
6916/6916 [==============================] - 0s 21us/step - loss: 1.1076 - acc: 0.5881 - val_loss: 1.4743 - val_acc: 0.4945
Epoch 125/200
6916/6916 [==============================] - 0s 20us/step - loss: 1.1000 - acc: 0.5998 - val_loss: 1.4828 - val_acc: 0.5003
Epoch 126/200
6916/6916 [==============================] - 0s 21us/step - loss: 1.0988 - acc: 0.6021 - val_loss: 1.4760 - val_acc: 0.5026
Epoch 127/200
6916/6916 [=========

In [ ]:
# saveModel('myModel.h5')
# loadModel('myModel.h5')

In [40]:
# Evaluating the model on the training and testing set
score = model.evaluate(features, targets)
print("\n Training Accuracy:", score[1])
score = model.evaluate(features_test, targets_test)
print("\n Testing Accuracy:", score[1])

8645/8645 [==============================] - 1s 75us/step

 Training Accuracy: 0.7367264314701683
961/961 [==============================] - 0s 73us/step

 Testing Accuracy: 0.5119667013527576


In [41]:
np.set_printoptions(suppress=True)
result_data=[]

def getGenre(index):
    if index == len(full_genres):
        return 'other'
    return full_genres[index][0]

for i in range(len(test_data.index.values)):
    index = test_data.index.values[i]+1
    result = []
    result.append(songs[index][0])
    result.append(songs[index][2])
    x = np.array([features_test[i]])
    for feature in x[0]:
        result.append(round(feature,3))
    y = np.array([targets_test[i]])
    prediction = model.predict(x)

    first = [-1,-1]
    second = [-2, -2]
    for j,guess in enumerate(prediction[0]):
        if guess > first[1]:
            second = first
            first = [j,guess]
        elif guess > second[1]:
            second = [j,guess]
    result.append(getGenre(first[0]))
    result.append(getGenre(second[0]))
    result.append(getGenre(songs[index][-1][0]))
    result_data.append(result)

result_array = np.array(result_data)

column_names_results= ['name','artist','danceability','energy','loudness','speechiness',
          'acousticness','instrumentalness','liveness','valence','tempo','1st guess', '2nd guess','actual']    

rf = pd.DataFrame(data=result_array[0:,0:],
                 columns=column_names_results)
rf[:50]
# with pd.option_context('display.max_rows', None, 'display.max_columns', 14):
#     display(rf)

,name,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,1st guess,2nd guess,actual
0,I Am,Jorja Smith,0.714,0.775,0.341,0.327,0.137,0.0,0.396,0.84,0.64,hip hop,downtempo,r&b
1,Eye 2 Eye,Huncho Jack,0.663,0.842,0.208,0.101,0.003,0.001,0.123,0.346,0.796,hip hop,edm,hip hop
2,How U Feel,Huncho Jack,0.72,0.725,0.199,0.052,0.008,0.0,0.137,0.202,0.715,hip hop,pop,hip hop
3,CINEMA 2,BROCKHAMPTON,0.856,0.712,0.318,0.062,0.163,0.031,0.123,0.774,0.645,house,edm,hip hop
4,"Piano Concerto No.3 In C, Op.26: 1. Andante - ...",Sergei Prokofiev,0.737,0.429,0.513,0.078,0.632,0.0,0.104,0.147,0.5,hip hop,r&b,classical
5,"Piano Concerto No.3 In C, Op.26: 2. Tema con v...",Sergei Prokofiev,0.312,0.26,0.905,0.038,0.967,0.863,0.084,0.146,0.734,jazz,soundtrack,classical
6,"Piano Concerto in G Major, M. 83: 2. Adagio assai",Maurice Ravel,0.341,0.189,1.028,0.038,0.981,0.856,0.328,0.095,0.66,classical,soundtrack,classical
7,"Gaspard de la nuit, M.55: Scarbo",Maurice Ravel,0.414,0.003,1.726,0.058,0.993,0.922,0.055,0.056,0.332,classical,downtempo,classical
8,"12 Sonates en trio, Op. 1, Sonate No. 2 in E M...",Antonio Vivaldi,0.9,0.365,0.32,0.235,0.147,0.185,0.179,0.136,0.675,hip hop,r&b,classical
9,Lush,Four Tet,0.626,0.758,0.2,0.243,0.022,0.0,0.37,0.642,0.7,hip hop,r&b,indietronica


In [42]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(danceability, energy,
     loudness, speechiness,
     acousticness, instrumentalness,
     liveness, valence,
     tempo):
    
    x = np.array([[danceability, energy, loudness, speechiness, acousticness, 
                  instrumentalness, liveness, valence,tempo]])
    prediction = model.predict(x)

    first = [-1,-1]
    second = [-2, -2]
    for j,guess in enumerate(prediction[0]):
        if guess > first[1]:
            second = first
            first = [j,guess]
        elif guess > second[1]:
            second = [j,guess]
            
    first_guess = getGenre(first[0])
    second_guess = getGenre(second[0])
    
    display(first_guess,second_guess)
    return first_guess

w = interactive(f, danceability=(0,1,0.1),
                energy=(0,1,0.1),
                loudness=(0,1,0.1),
                speechiness=(0,1,0.1),
                acousticness=(0,1,0.1),
                instrumentalness=(0,1,0.1),
                liveness=(0,1,0.1),
                valence=(0,1,0.1),
                tempo=(0,1,0.1))
display(w)


interactive(children=(FloatSlider(value=0.0, description='danceability', max=1.0), FloatSlider(value=0.0, description='energy', max=1.0), FloatSlider(value=0.0, description='loudness', max=1.0), FloatSlider(value=0.0, description='speechiness', max=1.0), FloatSlider(value=0.0, description='acousticness', max=1.0), FloatSlider(value=0.0, description='instrumentalness', max=1.0), FloatSlider(value=0.0, description='liveness', max=1.0), FloatSlider(value=0.0, description='valence', max=1.0), FloatSlider(value=0.0, description='tempo', max=1.0), Output()), _dom_classes=('widget-interact',))